In [1]:
import time
import arkouda as ak
import arachne as ar

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
ak.connect("n82", 5555)

connected to arkouda server tcp://*:5555


In [3]:
def get_gnp_random_graph(n, p, seed):
    G = ar.gnp_random_graph(n, p, create_using=ar.PropGraph, seed=seed)
    print(f"Built G(n,p) graph with {len(G):,} vertices and {G.size():,} edges.")

    return G

In [4]:
def build_subgraph(src, dst):
    akarray_src = ak.array(src)
    akarray_dst = ak.array(dst)

    H = ar.PropGraph()
    H.add_edges_from(akarray_src, akarray_dst)

    return H

In [5]:
def add_attributes(graph, num_node_lbls, num_edge_lbls, vals_per_lbl, seed):
    nodes = graph.nodes()
    edges = graph.edges()

    n = len(nodes)
    m = len(edges[0])

    node_attributes = []
    edge_attributes = []

    for i in range(num_node_lbls):
        node_attributes.append(ak.randint(0,vals_per_lbl,n,seed=seed*i,dtype=ak.int64))

    for i in range(num_edge_lbls):
        edge_attributes.append(ak.randint(0,vals_per_lbl,m,seed=seed*i,dtype=ak.int64))

    lbls_dict = {"lbl"+str(idx):attribute for idx, attribute in enumerate(node_attributes)}
    rels_dict = {"rel"+str(idx):attribute for idx, attribute in enumerate(edge_attributes)}

    node_dict = {"nodes" : nodes}
    edge_dict = {"src" : edges[0], "dst" : edges[1]}

    node_dict.update(lbls_dict)
    edge_dict.update(rels_dict)

    node_df = ak.DataFrame(node_dict)
    edge_df = ak.DataFrame(edge_dict)

    graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
    graph.load_node_attributes(node_df, node_column="nodes")

In [6]:
n = 60_000
p = 0.0005
seed = 42
num_node_lbls = 2
num_edge_lbls = 2
vals_per_lbl = 2

In [7]:
src = [0, 1, 2, 0]
dst = [1, 2, 0, 3]

In [8]:
G_no_attributes = get_gnp_random_graph(n, p, seed)

Built G(n,p) graph with 60,000 vertices and 1,800,598 edges.


In [9]:
H_no_attributes = build_subgraph(src, dst)

In [10]:
# Run structural subgraph isomorphism.
start = time.time()
results1 = ar.subgraph_isomorphism(G_no_attributes, H_no_attributes, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H_no_attributes):_} isos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_isomorphism(G_no_attributes, H_no_attributes, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H_no_attributes):_} isos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_isomorphism(G_no_attributes, H_no_attributes, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} isos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_isomorphism(G_no_attributes, H_no_attributes, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results4:_} isos with PS (count-only) took {end-start:.2f} seconds")

Finding 802_541 isos with SI took 1.04 seconds
Finding 802_541 isos with PS took 16.83 seconds
Finding 802_541 isos with SI (count-only) took 0.88 seconds
Finding 802_541 isos with PS (count-only) took 16.63 seconds


In [11]:
# Run structural subgraph monomorphism.
start = time.time()
results1 = ar.subgraph_monomorphism(G_no_attributes, H_no_attributes, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H_no_attributes):_} monos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_monomorphism(G_no_attributes, H_no_attributes, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H_no_attributes):_} monos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_monomorphism(G_no_attributes, H_no_attributes, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} monos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_monomorphism(G_no_attributes, H_no_attributes, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results4:_} monos with PS (count-only) took {end-start:.2f} seconds")

Finding 805_363 monos with SI took 0.98 seconds
Finding 805_363 monos with PS took 16.46 seconds
Finding 805_363 monos with SI (count-only) took 0.90 seconds
Finding 805_363 monos with PS (count-only) took 20.80 seconds


In [12]:
G = get_gnp_random_graph(n, p, seed)

Built G(n,p) graph with 60,000 vertices and 1,800,598 edges.


In [13]:
H = build_subgraph(src, dst)

In [14]:
add_attributes(G, num_node_lbls, num_edge_lbls, vals_per_lbl, seed)

In [15]:
add_attributes(H, num_node_lbls, num_edge_lbls, vals_per_lbl, seed)

In [16]:
# Run semantic subgraph isomorphism.
start = time.time()
results1 = ar.subgraph_isomorphism(G, H, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H):_} isos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_isomorphism(G, H, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H):_} isos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_isomorphism(G, H, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} isos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_isomorphism(G, H, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results4:_} isos with PS (count-only) took {end-start:.2f} seconds")

Finding 4 isos with SI took 0.20 seconds
Finding 4 isos with PS took 17.56 seconds
Finding 4 isos with SI (count-only) took 0.17 seconds
Finding 4 isos with PS (count-only) took 16.81 seconds


In [17]:
# Run semantic subgraph monomorphism.
start = time.time()
results1 = ar.subgraph_monomorphism(G, H, return_isos_as="vertices", algorithm_type="si")
end = time.time()
print(f"Finding {len(results1[0]) // len(H):_} monos with SI took {end-start:.2f} seconds")

start = time.time()
results2 = ar.subgraph_monomorphism(G, H, return_isos_as="vertices", algorithm_type="ps")
end = time.time()
print(f"Finding {len(results2[0]) // len(H):_} monos with PS took {end-start:.2f} seconds")

start = time.time()
results3 = ar.subgraph_monomorphism(G, H, return_isos_as="count", algorithm_type="si")
end = time.time()
print(f"Finding {results3:_} monos with SI (count-only) took {end-start:.2f} seconds")

start = time.time()
results4 = ar.subgraph_monomorphism(G, H, return_isos_as="count", algorithm_type="ps")
end = time.time()
print(f"Finding {results4:_} monos with PS (count-only) took {end-start:.2f} seconds")

Finding 4 monos with SI took 0.20 seconds
Finding 4 monos with PS took 16.48 seconds
Finding 4 monos with SI (count-only) took 0.15 seconds
Finding 4 monos with PS (count-only) took 19.88 seconds
